In [1]:
import json
import requests
import traceback
from datetime import datetime, timedelta
from playwright.async_api import async_playwright
from utils.calendar_utils import (
        generate_mock_calendar,
        format_calendar_data,
        convert_to_unified_format,
        find_overlapping_times
    )
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.output_parsers import ResponseSchema, StructuredOutputParser
from pprint import pprint
name = "John Doe"
email = "john@doe.com"
phone = "5109198404"

In [4]:
mock_calendar = generate_mock_calendar()
pprint(mock_calendar, indent=3)

{  'available_slots': [  {  'date': 'Wednesday, March 12, 2025',
                            'free_slots': [  {  'end': '10:00 AM',
                                                'start': '09:30 AM'},
                                             {  'end': '01:00 PM',
                                                'start': '11:00 AM'},
                                             {  'end': '05:00 PM',
                                                'start': '02:00 PM'}]},
                         {  'date': 'Thursday, March 13, 2025',
                            'free_slots': [  {  'end': '11:30 AM',
                                                'start': '09:30 AM'},
                                             {  'end': '02:30 PM',
                                                'start': '12:00 PM'},
                                             {  'end': '04:00 PM',
                                                'start': '03:30 PM'},
                                             { 

In [5]:
calendly_url = "https://calendly.com/robertjandali/30min"
url_parts = calendly_url.split('/')
profile_slug = url_parts[-2]
event_type_slug = url_parts[-1].split('?')[0]
id_url = f"https://calendly.com/api/booking/event_types/lookup?event_type_slug={event_type_slug}&profile_slug={profile_slug}"

# Get UUID
response = requests.get(id_url)
response.raise_for_status()
event_data = response.json()
uuid = event_data.get("uuid")

range_url = f"https://calendly.com/api/booking/event_types/{uuid}/calendar/range"
start_date = datetime.now()
end_date = start_date + timedelta(days=35)
params = {
    "timezone": "America/Los_Angeles",
    "diagnostics": "false",
    "range_start": start_date.strftime("%Y-%m-%d"),
    "range_end": end_date.strftime("%Y-%m-%d")
}

calendar_response = requests.get(range_url, params=params)
calendar_response.raise_for_status()
calendly_data = calendar_response.json()
calendly_formatted = format_calendar_data(calendly_data)
pprint(calendly_formatted, indent=3)

{  'available_dates': [  {  'date': 'March 12, 2025',
                            'iso_times': [  '2025-03-12T01:00:00-07:00',
                                            '2025-03-12T01:30:00-07:00',
                                            '2025-03-12T02:00:00-07:00',
                                            '2025-03-12T02:30:00-07:00',
                                            '2025-03-12T03:00:00-07:00',
                                            '2025-03-12T03:30:00-07:00',
                                            '2025-03-12T04:00:00-07:00',
                                            '2025-03-12T04:30:00-07:00',
                                            '2025-03-12T05:00:00-07:00',
                                            '2025-03-12T05:30:00-07:00',
                                            '2025-03-12T06:00:00-07:00',
                                            '2025-03-12T06:30:00-07:00',
                                            '2025-03-12T07:00:00-07:00

In [6]:
calendly_timezone = calendly_formatted.get("timezone", "America/Los_Angeles")
mock_calendar

{'week_of': 'March 12, 2025',
 'timezone': 'America/Los_Angeles',
 'available_slots': [{'date': 'Wednesday, March 12, 2025',
   'free_slots': [{'start': '09:30 AM', 'end': '10:00 AM'},
    {'start': '11:00 AM', 'end': '01:00 PM'},
    {'start': '02:00 PM', 'end': '05:00 PM'}]},
  {'date': 'Thursday, March 13, 2025',
   'free_slots': [{'start': '09:30 AM', 'end': '11:30 AM'},
    {'start': '12:00 PM', 'end': '02:30 PM'},
    {'start': '03:30 PM', 'end': '04:00 PM'},
    {'start': '04:30 PM', 'end': '05:00 PM'}]},
  {'date': 'Friday, March 14, 2025',
   'free_slots': [{'start': '09:00 AM', 'end': '10:00 AM'},
    {'start': '10:30 AM', 'end': '01:00 PM'},
    {'start': '01:30 PM', 'end': '02:30 PM'},
    {'start': '03:30 PM', 'end': '04:00 PM'}]},
  {'date': 'Monday, March 17, 2025',
   'free_slots': [{'start': '09:00 AM', 'end': '11:30 AM'},
    {'start': '12:00 PM', 'end': '01:00 PM'},
    {'start': '02:00 PM', 'end': '02:30 PM'},
    {'start': '03:30 PM', 'end': '04:00 PM'},
    {'star

In [7]:
unified_mock = convert_to_unified_format(mock_calendar, 'mock', calendly_timezone)
unified_mock

{'timezone': 'America/Los_Angeles',
 'available_days': [{'date': 'March 12, 2025',
   'day_of_week': 'Wednesday',
   'available_times': ['09:30 AM',
    '11:00 AM',
    '11:30 AM',
    '12:00 PM',
    '12:30 PM',
    '02:00 PM',
    '02:30 PM',
    '03:00 PM',
    '03:30 PM',
    '04:00 PM',
    '04:30 PM']},
  {'date': 'March 13, 2025',
   'day_of_week': 'Thursday',
   'available_times': ['09:30 AM',
    '10:00 AM',
    '10:30 AM',
    '11:00 AM',
    '12:00 PM',
    '12:30 PM',
    '01:00 PM',
    '01:30 PM',
    '02:00 PM',
    '03:30 PM',
    '04:30 PM']},
  {'date': 'March 14, 2025',
   'day_of_week': 'Friday',
   'available_times': ['09:00 AM',
    '09:30 AM',
    '10:30 AM',
    '11:00 AM',
    '11:30 AM',
    '12:00 PM',
    '12:30 PM',
    '01:30 PM',
    '02:00 PM',
    '03:30 PM']},
  {'date': 'March 17, 2025',
   'day_of_week': 'Monday',
   'available_times': ['09:00 AM',
    '09:30 AM',
    '10:00 AM',
    '10:30 AM',
    '11:00 AM',
    '12:00 PM',
    '12:30 PM',
    '02

In [8]:
unified_calendly = convert_to_unified_format(calendly_formatted, 'calendly')
unified_calendly

{'timezone': 'America/Los_Angeles',
 'available_days': [{'date': 'March 12, 2025',
   'day_of_week': 'Wednesday',
   'available_times': ['01:00 AM',
    '01:30 AM',
    '02:00 AM',
    '02:30 AM',
    '03:00 AM',
    '03:30 AM',
    '04:00 AM',
    '04:30 AM',
    '05:00 AM',
    '05:30 AM',
    '06:00 AM',
    '06:30 AM',
    '07:00 AM',
    '07:30 AM',
    '08:00 AM',
    '08:30 AM',
    '09:00 AM',
    '09:30 AM',
    '10:00 AM',
    '10:30 AM',
    '11:00 AM',
    '11:30 AM',
    '12:00 PM',
    '12:30 PM',
    '01:00 PM',
    '01:30 PM',
    '02:00 PM',
    '02:30 PM',
    '03:00 PM',
    '03:30 PM',
    '04:00 PM',
    '04:30 PM',
    '05:00 PM',
    '05:30 PM',
    '06:00 PM',
    '06:30 PM',
    '07:00 PM',
    '07:30 PM',
    '08:00 PM',
    '08:30 PM',
    '09:00 PM',
    '09:30 PM',
    '10:00 PM',
    '10:30 PM',
    '11:00 PM']},
  {'date': 'March 13, 2025',
   'day_of_week': 'Thursday',
   'available_times': ['12:00 AM',
    '12:30 AM',
    '01:00 AM',
    '01:30 AM',
   

In [9]:
overlapping_calendar = find_overlapping_times(unified_mock, unified_calendly)
overlapping_calendar

{'timezone': 'America/Los_Angeles',
 'available_days': [{'date': 'March 12, 2025',
   'day_of_week': 'Wednesday',
   'available_times': ['09:30 AM',
    '11:00 AM',
    '11:30 AM',
    '12:00 PM',
    '12:30 PM',
    '02:00 PM',
    '02:30 PM',
    '03:00 PM',
    '03:30 PM',
    '04:00 PM',
    '04:30 PM']},
  {'date': 'March 13, 2025',
   'day_of_week': 'Thursday',
   'available_times': ['09:30 AM',
    '10:00 AM',
    '10:30 AM',
    '11:00 AM',
    '12:00 PM',
    '12:30 PM',
    '01:00 PM',
    '01:30 PM',
    '02:00 PM',
    '03:30 PM',
    '04:30 PM']},
  {'date': 'March 14, 2025',
   'day_of_week': 'Friday',
   'available_times': ['09:00 AM',
    '09:30 AM',
    '10:30 AM',
    '11:00 AM',
    '11:30 AM',
    '12:00 PM',
    '12:30 PM',
    '01:30 PM',
    '02:00 PM',
    '03:30 PM']},
  {'date': 'March 17, 2025',
   'day_of_week': 'Monday',
   'available_times': ['09:00 AM',
    '09:30 AM',
    '10:00 AM',
    '10:30 AM',
    '11:00 AM',
    '12:00 PM',
    '12:30 PM',
    '02

In [10]:
from utils.calendar_utils import convert_to_unified_format, find_overlapping_times
calendly_timezone = calendly_formatted.get("timezone", "America/Los_Angeles")
unified_mock = convert_to_unified_format(mock_calendar, 'mock', calendly_timezone)
unified_calendly = convert_to_unified_format(calendly_formatted, 'calendly')
overlapping_calendar = find_overlapping_times(unified_mock, unified_calendly)
print(overlapping_calendar)

{'timezone': 'America/Los_Angeles', 'available_days': [{'date': 'March 12, 2025', 'day_of_week': 'Wednesday', 'available_times': ['09:30 AM', '11:00 AM', '11:30 AM', '12:00 PM', '12:30 PM', '02:00 PM', '02:30 PM', '03:00 PM', '03:30 PM', '04:00 PM', '04:30 PM']}, {'date': 'March 13, 2025', 'day_of_week': 'Thursday', 'available_times': ['09:30 AM', '10:00 AM', '10:30 AM', '11:00 AM', '12:00 PM', '12:30 PM', '01:00 PM', '01:30 PM', '02:00 PM', '03:30 PM', '04:30 PM']}, {'date': 'March 14, 2025', 'day_of_week': 'Friday', 'available_times': ['09:00 AM', '09:30 AM', '10:30 AM', '11:00 AM', '11:30 AM', '12:00 PM', '12:30 PM', '01:30 PM', '02:00 PM', '03:30 PM']}, {'date': 'March 17, 2025', 'day_of_week': 'Monday', 'available_times': ['09:00 AM', '09:30 AM', '10:00 AM', '10:30 AM', '11:00 AM', '12:00 PM', '12:30 PM', '02:00 PM', '03:30 PM', '04:30 PM']}, {'date': 'March 18, 2025', 'day_of_week': 'Tuesday', 'available_times': ['11:00 AM', '12:00 PM', '12:30 PM', '01:00 PM', '01:30 PM', '02:00 

In [11]:
response_schema = ResponseSchema(
    name="suggested_time",
    description="The suggested meeting time in ISO 8601 format",
    type="string"
)
        
        # Create the parser
parser = StructuredOutputParser.from_response_schemas([response_schema])
format_instructions = parser.get_format_instructions()

In [12]:
from prompts.scheduling_prompts import scheduling_prompt
prompt_template = scheduling_prompt(overlapping_calendar, format_instructions)
prompt_template

ChatPromptTemplate(input_variables=['format_instructions', 'overlapping_availability'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['format_instructions', 'overlapping_availability'], input_types={}, partial_variables={}, template='\n    You are a scheduling assistant. Please analyze the following available meeting times and suggest the best option.\n\n    Overlapping Available Times:\n    {overlapping_availability}\n\n    Find the best meeting time from these overlapping time slots at the earliest reasonable business hour.\n\n    {format_instructions}\n\n    Note: The suggested_time MUST be in this exact format:\n    "2025-03-05T09:30:00-08:00?month=2025-03&date=2025-03-05"\n    - Include ISO 8601 datetime with timezone offset (-08:00 for PST)\n    - Include query parameters for month and date\n    '), additional_kwargs={})])

In [13]:
# Get suggested time from LLM
messages = prompt_template.format_messages(
    overlapping_availability=json.dumps(overlapping_calendar, indent=2),
    format_instructions=format_instructions
)

In [14]:
formatted_prompt = "\n".join(message.content for message in messages)
pprint(formatted_prompt)

('\n'
 '    You are a scheduling assistant. Please analyze the following available '
 'meeting times and suggest the best option.\n'
 '\n'
 '    Overlapping Available Times:\n'
 '    {\n'
 '  "timezone": "America/Los_Angeles",\n'
 '  "available_days": [\n'
 '    {\n'
 '      "date": "March 12, 2025",\n'
 '      "day_of_week": "Wednesday",\n'
 '      "available_times": [\n'
 '        "09:30 AM",\n'
 '        "11:00 AM",\n'
 '        "11:30 AM",\n'
 '        "12:00 PM",\n'
 '        "12:30 PM",\n'
 '        "02:00 PM",\n'
 '        "02:30 PM",\n'
 '        "03:00 PM",\n'
 '        "03:30 PM",\n'
 '        "04:00 PM",\n'
 '        "04:30 PM"\n'
 '      ]\n'
 '    },\n'
 '    {\n'
 '      "date": "March 13, 2025",\n'
 '      "day_of_week": "Thursday",\n'
 '      "available_times": [\n'
 '        "09:30 AM",\n'
 '        "10:00 AM",\n'
 '        "10:30 AM",\n'
 '        "11:00 AM",\n'
 '        "12:00 PM",\n'
 '        "12:30 PM",\n'
 '        "01:00 PM",\n'
 '        "01:30 PM",\n'
 '     

In [15]:
import os
llm = ChatOpenAI(
    model_name="gpt-4o-mini",
    temperature=0,
    openai_api_key=os.getenv("OPENAI_API_KEY")
)
response = llm.invoke(messages)
parsed_response = parser.parse(response.content)
suggested_time = parsed_response['suggested_time']
print(f"LLM suggested time: {suggested_time}")

/tmp/ipykernel_756508/2218657616.py:2: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(


LLM suggested time: 2025-03-12T09:30:00-08:00?month=2025-03&date=2025-03-12


In [16]:
import pytz
from datetime import datetime
suggested_time = parsed_response['suggested_time']
clean_time_str = suggested_time.split('?')[0]
suggested_time_dt = datetime.fromisoformat(clean_time_str)
# No need for timezone conversion since it's already in PST
print(suggested_time)
print(f"LLM suggested time: {suggested_time_dt.strftime('%I:%M %p')} PST")

2025-03-12T09:30:00-08:00?month=2025-03&date=2025-03-12
LLM suggested time: 09:30 AM PST


In [17]:
iso_datetime = suggested_time.split('?')[0] if '?' in suggested_time else suggested_time
suggested_time

'2025-03-12T09:30:00-08:00?month=2025-03&date=2025-03-12'

In [18]:

base_path = '/'.join(calendly_url.split('?')[0].split('/')[:-1]) if calendly_url.endswith('/') else '/'.join(calendly_url.split('?')[0].split('/'))
final_url = f"{base_path}/{suggested_time}"
print(f"Created booking URL: {final_url}")

Created booking URL: https://calendly.com/robertjandali/30min/2025-03-12T09:30:00-08:00?month=2025-03&date=2025-03-12


Using Anchor Browser with custom session

In [109]:
anchor_api_key = os.getenv("ANCHOR_API_KEY")
url = "https://api.anchorbrowser.io/api/sessions"
payload = {
                    "adblock_config": {
                        "active": False,
                        "popup_blocking_active": False
                    },
                    "captcha_config": {"active": True},
                    "headless": False,
                    "proxy_config": {
                        "type": "anchor_residential",
                        "active": True
                    },
                    "recording": {"active": True},
                    "profile": {
                        "name": "my-profile",
                        "persist": True,
                        "store_cache": True
                    },
                    "viewport": {
                        "width": 1440,
                        "height": 900
                    },
                    "timeout": 10,
                    "idle_timeout": 3
                }
                

headers = {
    "anchor-api-key": anchor_api_key,
    "Content-Type": "application/json"
}

response = requests.request("POST", url, json=payload, headers=headers)
session_id = response.json()['id']
anchor_url = f"wss://connect.anchorbrowser.io?apiKey={anchor_api_key}&sessionId={session_id}"

async with async_playwright() as p:
    browser = await p.chromium.connect_over_cdp(anchor_url)
    context = browser.contexts[0]
    ai = context.service_workers[0]
    page = context.pages[0]
                        
    await page.goto(final_url)
                        
    task = f"~ Fill out the form with name {name}, email {email}, phone {phone}, and click the Schedule Event button. Wait until the confirmation page is loaded or a success message appears."
    result = await ai.evaluate(task)
    print(result)
                        
    await browser.close()
    
delete_url = f"https://api.anchorbrowser.io/api/sessions/{session_id}"
requests.request("DELETE", delete_url, headers=headers)

Filled out the form with name John Doe, email john@doe.com, phone 5109198404, and clicked the Schedule Event button. The confirmation page is loaded.


<Response [200]>

Using Anchor Browser with default session

In [101]:
anchor_api_key = os.getenv("ANCHOR_API_KEY")

anchor_url = f"wss://connect.anchorbrowser.io?apiKey={anchor_api_key}"

async with async_playwright() as p:
    browser = await p.chromium.connect_over_cdp(anchor_url)
    context = browser.contexts[0]
    ai = context.service_workers[0]
    page = context.pages[0]
                        
    await page.goto(final_url)
                        
    task = f"~ Fill out the form with name {name}, email {email}, phone {phone}, and click the Schedule Event button. Wait until the confirmation page is loaded or a success message appears."
    result = await ai.evaluate(task)
    print(result)
                        
    await browser.close()

I have filled out the form with name John Doe, email john@doe.com, phone 5109198404, and clicked the Schedule Event button. Waited for the confirmation page to load. The event has been scheduled.


Using Stagehand

In [69]:
from stagehand.schemas import ActOptions
from stagehand.client import Stagehand

import asyncio
import os
from dotenv import load_dotenv
import nest_asyncio
nest_asyncio.apply()
from stagehand.config import StagehandConfig

# Load environment variables
load_dotenv()

async def init_stagehand():
    try:
        # Initialize Stagehand with configuration
        stagehand = StagehandConfig(
            model_name='gpt-4o-mini',
            env='BROWSERBASE',
            api_key=os.getenv('BROWSERBASE_API_KEY'),
            browserbase_session_create_params={
                'project_id': os.getenv('BROWSERBASE_PROJECT_ID'),
                'browser_settings': {
                    'solve_captchas': True
                }
            }
        )

        # Initialize the browser
        await stagehand.init()

        # Get the page object
        page = stagehand.page

        # Navigate to the Calendly page
        await page.goto(final_url)

        # Fill out the form using act
        await page.act({
            'action': "enter %name% into the Name field, enter %email% into the Email field, enter %phone% into the Phone Number field and click on the Schedule Event Button",
            'variables': {
                'name': "John Doe",
                'email': "john.doe@example.com",
                'phone': "+15109198404",
            }
        })

        # Close the browser
        await stagehand.close()

    except Exception as error:
        print(f"Error occurred: {error}")

if __name__ == "__main__":
    # Run the async function
    asyncio.run(init_stagehand())

Error occurred: 'StagehandConfig' object has no attribute 'init'


In [71]:
import subprocess
import os

# Path to your stagehand.ts file
file_path = os.path.join(os.getcwd(), 'stagehand', 'stagehand.ts')

# Execute the TypeScript file using ts-node
try:
    result = subprocess.run(['npx', 'ts-node', file_path], 
                           capture_output=True, 
                           text=True, 
                           check=True)
    print(result.stdout)
except subprocess.CalledProcessError as e:
    print(f"Error: {e}")
    print(f"stderr: {e.stderr}")

2025-03-12T00:47:28.884Z::[stagehand:init] launching local browser {"headless":{"value":"false","type":"boolean"}}
2025-03-12T00:47:29.715Z::[stagehand:init] local browser started successfully. 
2025-03-12T00:47:33.137Z::[stagehand:act] running act {"action":{"value":"enter %name% into the Name field, enter %email% into the Email field, enter %phone% into the Phone Number field and click on the Schedule Event Button","type":"string"},"requestId":{"value":"5czm7l9araj","type":"string"},"modelName":{"value":"gpt-4o-mini","type":"string"}}
2025-03-12T00:47:33.156Z::[stagehand:action] running / continuing action {"action":{"value":"enter %name% into the Name field, enter %email% into the Email field, enter %phone% into the Phone Number field and click on the Schedule Event Button","type":"string"},"pageUrl":{"value":"https://calendly.com/robertjandali/30min/2025-03-10T09:30:00-08:00?month=2025-03&date=2025-03-10","type":"string"}}
2025-03-12T00:47:33.156Z::[stagehand:action] processing DOM

In [1]:
from openai import OpenAI
client = OpenAI()

completion = client.chat.completions.create(
    model="gpt-4o-search-preview",
    web_search_options={},
    messages=[
        {
            "role": "user",
            "content": "What was a positive news story from today?",
        }
    ],
)

print(completion.choices[0].message.content)

On March 11, 2025, Indonesian authorities successfully rescued a critically endangered two-month-old male Sumatran elephant that had become separated from its mother in a palm oil plantation. The calf was found in Riau province on Sumatra island after residents alerted the authorities. Sumatran elephants are on the brink of extinction, with only about 2,400-2,800 individuals remaining worldwide. This rescue represents a significant effort in the conservation of this endangered species. ([straitstimes.com](https://www.straitstimes.com/world/while-you-were-sleeping-5-stories-you-might-have-missed-march-12-2025?utm_source=chatgpt.com)) 


In [6]:
from playwright.sync_api import sync_playwright
from playwright.async_api import async_playwright

async with async_playwright() as p:
    browser = await p.chromium.launch(
        headless=False,
        chromium_sandbox=True,
        env={},
        args=[
            "--disable-extensions",
            "--disable-file-system"
        ]
    )
    page = browser.new_page()
    page.set_viewport_size({"width": 1024, "height": 768})
    page.goto("https://bing.com")

    page.wait_for_timeout(10000)

TargetClosedError: BrowserType.launch: Target page, context or browser has been closed
Browser logs:

╔════════════════════════════════════════════════════════════════════════════════════════════════╗
║ Looks like you launched a headed browser without having a XServer running.                     ║
║ Set either 'headless: true' or use 'xvfb-run <your-playwright-app>' before running Playwright. ║
║                                                                                                ║
║ <3 Playwright Team                                                                             ║
╚════════════════════════════════════════════════════════════════════════════════════════════════╝
Call log:
  - <launching> /home/zain/.cache/ms-playwright/chromium-1155/chrome-linux/chrome --disable-field-trial-config --disable-background-networking --disable-background-timer-throttling --disable-backgrounding-occluded-windows --disable-back-forward-cache --disable-breakpad --disable-client-side-ph

In [2]:
from openai import OpenAI
client = OpenAI()

response = client.responses.create(
    model="computer-use-preview",
    tools=[{
        "type": "computer_use_preview",
        "display_width": 1024,
        "display_height": 768,
        "environment": "browser" # other possible values: "mac", "windows", "ubuntu"
    }],
    input=[
        {
            "role": "user",
            "content": "Check the latest OpenAI news on bing.com."
        }
        # Optional: include a screenshot of the initial state of the environment
        # {
        #     type: "input_image",
        #     image_url: f"data:image/png;base64,{screenshot_base64}"
        # }
    ],
    reasoning={
        "generate_summary": "concise",
    },
    truncation="auto"
)

print(response.output)

NotFoundError: Error code: 404 - {'error': {'message': 'The model `computer-use-preview-2025-03-11` does not exist or you do not have access to it.', 'type': 'invalid_request_error', 'param': None, 'code': 'model_not_found'}}